## Problema 1 Tarea 5 Modelos Estadísticos ##

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

sys.path.append(".")
from LinReg import mlr # Regresión Múltiple
from LinReg import slr # Regresión Simple

dataP1=pd.read_csv('DatosT5P1.csv')

## Inciso a) ##
mlrP1=mlr(dataP1)
mlrP1.OLS()
mlrP1.summary()

                       Results: Ordinary least squares
Model:                  OLS              Adj. R-squared (uncentered): 0.983   
Dependent Variable:     Y                AIC:                         135.5357
Date:                   2021-04-21 18:36 BIC:                         137.9735
No. Observations:       25               Log-Likelihood:              -65.768 
Df Model:               2                F-statistic:                 734.8   
Df Residuals:           23               Prob (F-statistic):          1.44e-21
R-squared (uncentered): 0.985            Scale:                       12.267  
------------------------------------------------------------------------------------
           Coef.        Std.Err.         t          P>|t|        [0.025       0.975]
------------------------------------------------------------------------------------
x1         1.7079         0.1775       9.6207       0.0000       1.3407       2.0751
x2         0.0161         0.0038       4.2593       

### Calculamos el la correlación y el número de condición ###

In [2]:

X=np.column_stack([dataP1['X1'],dataP1['X2']])
X_means=np.mean(X,axis=0)
X_center=X-X_means
X_norms=np.linalg.norm(X_center,axis=0)
X_std = X_center/X_norms
Z=X_std.T@X_std
invZ=np.linalg.inv(Z)

## inciso b) ##
print('Correlación X1 y X2: %.4f' %(Z[0][1]))

## inciso c) ##
print('VIF(b1)= %.4f y VIF(b2)= %.4f' %(invZ[0][0],invZ[1][1]))

## inciso d) ##
eigs = np.linalg.eigvals(Z)
condition_number = np.sqrt(eigs.max() / eigs.min())
print('Cond. num= %.4f' %(condition_number))

Correlación X1 y X2: 0.8242
VIF(b1)= 3.1185 y VIF(b2)= 3.1185
Cond. num= 3.2214


## Problema 2 Tarea 5 Modelos Estadísticos ##


In [ ]:
dataP2=pd.read_csv('DatosT5P2.csv')

## inciso a) ##
slrP2=slr(dataP2)
slrP2.OLS()
slrP2.summary()
slrP2.plot_regression()



In [ ]:
## inciso b) ##
slrP2.resid_vs_previous_resid_plot() # Gráfica


In [ ]:
import statsmodels.api as sm

## inciso c) ##

Y=np.asarray(slrP2.y)
X=np.asarray(slrP2.x)
e=slrP2.results.resid
et1=e[1:]
et2=np.asarray(e[:-1])

# Regresión para estimar rho
results1 = sm.OLS(et1,et2).fit()
print(results1.summary2())

In [ ]:
rho=results1.params[0]

# Modelo de Cochrane-Orcutt
Y_new=Y[1:]-rho*Y[:-1]
X_new=X[1:]-rho*X[:-1]
X_new_design=np.column_stack([(1-rho)*np.ones(len(X_new)),X_new])

results_Coch_Or=sm.OLS(Y_new,X_new_design).fit()
print(results_Coch_Or.summary2())

In [ ]:
## inciso d) ##

# Grafica
residuals_Coch_Or = results_Coch_Or.resid
plt.figure(figsize=(10,5))
plt.scatter(residuals_Coch_Or[:-1],residuals_Coch_Or[1:])
plt.xlabel('Residuals_i')
plt.ylabel('Residuals_i-1')
plt.title("Residuals vs Previous Residuals")


## Problema 3 Tarea 5 Modelos Estadísticos ##

In [ ]:
mlrP1.residual_analysis

In [ ]:
## inciso a) ##

mlrP1.residual_analysis()
h_ii=np.asarray(mlrP1.residual_analysis['hat_diag'])
leverange_index=np.where(h_ii>2*mlrP1.p/mlrP1.n)[0]
print('Puntos palanca:')
print(leverange_index+np.ones(len(leverange_index)))



In [ ]:
## inciso b) ##
from scipy.stats import beta, t, f
dfn=mlrP1.p
dfd=mlrP1.n-mlrP1.p
cutoff_DCook=4/mlrP1.n
D_Cook=np.asarray(mlrP1.residual_analysis['cooks_d'])
D_Cook_index=np.where(D_Cook>cutoff_DCook)[0]
print('Puntos influyentes en la respuesta (D-Cook):')
print(D_Cook_index+np.ones(len(D_Cook_index)))


In [ ]:

## inciso c) ##
DFBETA_1=np.asarray(mlrP1.residual_analysis['dfb_x1'])
DFBETA_2=np.asarray(mlrP1.residual_analysis['dfb_x2'])
DFFITS=np.asarray(mlrP1.residual_analysis['dffits'])
corte_DFBETA=2/np.sqrt(mlrP1.n)
corte_DFFITS=2*np.sqrt(mlrP1.p/mlrP1.n)


print('OBSERVACION 9:')
print('DFBETA_1: %.4f, DFBETA_2: %.4f, DFFITS: %.4f' %(DFBETA_1[8],DFBETA_2[8],DFFITS[8]))

print('\nOBSERVACION 22:')
print('DFBETA_1: %.4f, DFBETA_2: %.4f, DFFITS: %.4f' %(DFBETA_1[21],DFBETA_2[21],DFFITS[21]))

print('\nCorte DFBETA: %.4f' %(corte_DFBETA))
print('\nCorte DFFITS: %.4f' %(corte_DFFITS))

In [ ]:
influence_DFBETA_1=np.where(abs(DFBETA_1)>corte_DFBETA)[0]
influence_DFBETA_2=np.where(abs(DFBETA_2)>corte_DFBETA)[0]
influence_DFFITS=np.where(abs(DFFITS)>corte_DFFITS)[0]
print('Observaciones descartables DFBETA1:')
print(influence_DFBETA_1+np.ones(len(influence_DFBETA_1)))
print('\nObservaciones descartables DFBETA2:')
print(influence_DFBETA_2+np.ones(len(influence_DFBETA_2)))
print('\nObservaciones descartables DFFITS:')
print(influence_DFFITS+np.ones(len(influence_DFFITS)))



In [ ]:
## inciso e) ##
mlrP1.resid_density_plot()
mlrP1.resid_qq_plot()


## Problema 4 Tarea 5 Modelos Estadísticos ##


In [ ]:
critical_value=t.ppf(1-mlrP1.alpha/2,mlrP1.n-mlrP1.p-1)
SSE = sum(mlrP1.results.resid**2)
s2_without_i = (SSE-mlrP1.results.resid**2/(1-h_ii))/(mlrP1.n-mlrP1.p-1)
studentized_residuals = mlrP1.results.resid/(np.sqrt(s2_without_i*(1-h_ii)))
outer_studentized=np.where(abs(studentized_residuals)>abs(critical_value))
print('Valores Atípicos de los residuos estudentizados:')
print(outer_studentized+np.ones(len(outer_studentized)))


In [ ]:
dataP4=pd.read_csv('DatosT5P4.csv')

X_Dum=dataP4['X'].to_numpy()

dataColonia=dataP4.loc[dataP4['Z1']==1]
dataColonia.drop(['Z1', 'Z2'], axis=1)
dataCentroCom=dataP4.loc[dataP4['Z2']==1]
dataCentroCom.drop(['Z1', 'Z2'], axis=1)
dataCentro=dataP4.loc[dataP4['Z1']+dataP4['Z2']==0]
dataCentro.drop(['Z1', 'Z2'], axis=1)

## inciso a) ##

slrColonia=slr(dataColonia)
slrCentroCom=slr(dataCentroCom)
slrCentro=slr(dataCentro)

slrColonia.OLS()
slrCentroCom.OLS()
slrCentro.OLS()


In [ ]:
slrColonia.summary()

In [ ]:
slrCentroCom.summary()

In [ ]:
slrCentro.summary()

In [ ]:
## iniciso b) ##

x0=np.linspace(min(X_Dum),max(X_Dum), 100)
XX0 = np.column_stack([np.ones(100),x0])

plt.plot(x0,slrColonia.results.predict(XX0),color='r',label='Colonia')
plt.plot(x0,slrCentroCom.results.predict(XX0),color='g',label='CentroCom')
plt.plot(x0,slrCentro.results.predict(XX0),color='b',label='Centro')
plt.scatter(slrColonia.x,slrColonia.y,color='r',marker='o')
plt.scatter(slrCentroCom.x,slrCentroCom.y,color='g',marker='*')
plt.scatter(slrCentro.x,slrCentro.y,color='b',marker='^')
plt.legend()



In [ ]:

## inciso b) ##
X_dum=np.column_stack([np.ones(dataP4.shape[0]),dataP4['X'],dataP4['Z1'],dataP4['Z2']])
results_Dum=sm.OLS(dataP4['Y'],X_dum).fit()
results_Dum.summary2()

In [ ]:
covMatriz=results_Dum.scale*np.linalg.inv(X_dum.T@X_dum)
dif1=-6.8638-21.5100
std_dif1=np.sqrt(4.7705**2+4.0651**2-2*covMatriz[2][3])

print('(Colonia-CentroCom): %.4f, std(Colonia-CentroCom):%.4f, t-valor: %.4f' %(dif1,std_dif1,dif1/std_dif1))
print('\n(Colonia-Centro): %.4f, std(Colonia-Centro): %.4f' %(-6.8638,4.7705))
print('\n(CentroCom-Centro): %.4f, std(CentroCom-Centro): %.4f' %(21.5100,4.0651))

In [ ]:
## inciso e) ##

## inciso b) ##
X_dum_pendiente=np.column_stack([np.ones(dataP4.shape[0]),dataP4['X'],dataP4['Z1'],dataP4['Z2'],dataP4['X']*dataP4['Z1'],dataP4['X']*dataP4['Z2']])
results_Dum_pendiente=sm.OLS(dataP4['Y'],X_dum_pendiente).fit()
results_Dum_pendiente.summary2()

In [ ]:
covMatriz_Pendiente=results_Dum_pendiente.scale*np.linalg.inv(X_dum_pendiente.T@X_dum_pendiente)
dif2=0.0336+0.0581
std_dif2=np.sqrt(0.1382**2+0.0932**2-2*covMatriz_Pendiente[4][5])
print('(Colonia-CentroCom): %.4f, std(Colonia-CentroCom):%.4f, t-valor: %.4f' %(dif2,std_dif2,dif2/std_dif2))
print('\n(Colonia-Centro): %.4f, std(Colonia-Centro): %.4f' %(0.0336,0.1382))
print('\n(CentroCom-Centro): %.4f, std(CentroCom-Centro): %.4f' %(-0.0581,0.0932))